In [20]:
import pandas as pd 
from athena_to_pd import athena_query
import sqlite3
#By Book
from pathlib import Path
import os
import awswrangler as wr 
import boto3
from datetime import datetime

In [ ]:
pwd = os.getcwd()

PROFILE = "vej-book"
DATABASE = "vtnlake_dev_curated_finance" 
#SQLITE_PATH = "../vtnlake_onedrive.db"
SQLITE_PATH = rf'D:\Work\SQLite\vtnlake_onedrive.db' # Book's Labtop
#SQLITE_PATH = rf'E:\Work\TradeSquare\SQlite Test\29102024\vtnlake_onedrive.db' # Book's PC

In [ ]:
date_filter = athena_query(PROFILE, DATABASE,"""
    select 
        min(date_format(parse_datetime(_file_updated_date,'yyyy-MM-dd''T''HH:mm:ss''Z'),'%Y-%m-%d %H:%i:%s')) AS min_date,
        max(date_format(parse_datetime(_file_updated_date,'yyyy-MM-dd''T''HH:mm:ss''Z'),'%Y-%m-%d %H:%i:%s')) AS max_date
        
    from gl_line
""")

# SQLite
# """
#     MIN(REPLACE(SUBSTR(file_name, 1, 19), 'T', ' ')) AS min_date,
#     MAX(REPLACE(SUBSTR(file_name, 1, 19), 'T', ' ')) AS max_date
# """

"\n    MIN(REPLACE(SUBSTR(file_name, 1, 19), 'T', ' ')) AS min_date,\n    MAX(REPLACE(SUBSTR(file_name, 1, 19), 'T', ' ')) AS max_date\n"

In [23]:
df_date_filter = pd.DataFrame(date_filter, columns = date_filter.columns)
min_date = df_date_filter.iloc[0, 0]
max_date = df_date_filter.iloc[0, 1]

print("Min Date Filter: ", min_date)
print("Max Date Filter: ", max_date)


Min Date Filter:  2024-01-11 03:32:39
Max Date Filter:  2024-10-17 07:40:41


In [24]:
'''
Select DISTINCT script for Athena (only PK)
'''

# Read the SQL template file (Book's PC)
with open(rf"{pwd}/sql_template/gl_line/select_dis_gl_line_athena.sql", "r") as file:
    query_template = file.read()

# Substitute placeholders with actual values
query_gl_line = query_template.format(min_date=min_date, max_date=max_date)

# Run the query with wr
select_gl_line_athena = wr.athena.read_sql_query(sql=query_gl_line, 
        database=DATABASE,
        boto3_session=boto3.session.Session(profile_name=PROFILE, region_name='ap-southeast-1')                      
                              )

##---------------------------------##
#global variables for sqlite
TABLE = "gl_line_full"

'''
Select DISTINCT script for SQLite (only PK)
'''

# Read the SQL template file
with open(rf"{pwd}/sql_template/gl_line/select_dis_gl_line_sqlite.sql", "r") as file:
    query_template = file.read()

# Substitute placeholders with actual values
SQLITE_QUERY = query_template.format(
    TABLE=TABLE,
    min_date=min_date,
    max_date=max_date
)    
# Execute the query
conn = sqlite3.connect(SQLITE_PATH)
df_select_sqlite = pd.read_sql_query(SQLITE_QUERY, conn).add_suffix('_sqlite')
conn.close()
select_gl_line_sqlite = df_select_sqlite


In [10]:
select_gl_line_athena.describe(include='all')

,invoice_no_receipt_no,invoice_receipt_line_no,order_item_id,order_item_sub_id
count,2948296,2948296,2948296,2948296
unique,212855,114051,1,1
top,CI67008915,42,,
freq,3951,14118,2948296,2948296


In [25]:
select_gl_line_athena.describe(include='all')

,invoice_no_receipt_no,invoice_receipt_line_no,order_item_id,order_item_sub_id
count,2948296,2948296,2948296,2948296
unique,212855,114051,1,1
top,CI67008915,42,0,0
freq,3951,14118,2948296,2948296


In [ ]:
# #select gl_line_athena where order_item_id is null or blank
# select_gl_line_athena_null = select_gl_line_athena[select_gl_line_athena['order_item_id'].isnull()]
# select_gl_line_athena_blank = select_gl_line_athena[select_gl_line_athena['order_item_id'] == '']

In [30]:
#sqlite
select_gl_line_sqlite.sort_values(by=['invoice_no_receipt_no_sqlite', 'invoice_receipt_line_no_sqlite', 'order_item_id_sqlite', 'order_item_sub_id_sqlite'], inplace=True)
select_gl_line_sqlite.reset_index(drop=True, inplace=True)
#athena
select_gl_line_athena.sort_values(by=['invoice_no_receipt_no', 'invoice_receipt_line_no', 'order_item_id', 'order_item_sub_id'], inplace=True)
select_gl_line_athena.reset_index(drop=True, inplace=True)

#Concat dataframe for Reconcilation
df_reconcile = pd.concat([select_gl_line_sqlite, select_gl_line_athena], axis=1)

#check status if suffix sqlite is match with athena using lambda for each column and row
df_reconcile['invoice_no_receipt_no_status'] = df_reconcile.apply(lambda x: 'Match' if x['invoice_no_receipt_no_sqlite'] == x['invoice_no_receipt_no'] else 'Mismatch', axis=1)
df_reconcile['invoice_receipt_line_no_status'] = df_reconcile.apply(lambda x: 'Match' if x['invoice_receipt_line_no_sqlite'] == x['invoice_receipt_line_no'] else 'Mismatch', axis=1)
df_reconcile['order_item_id_status'] = df_reconcile.apply(lambda x: 'Match' if x['order_item_id_sqlite'] == x['order_item_id'] else 'Mismatch', axis=1)
df_reconcile['order_item_sub_id_status'] = df_reconcile.apply(lambda x: 'Match' if x['order_item_sub_id_sqlite'] == x['order_item_sub_id'] else 'Mismatch', axis=1)
df_reconcile['status'] = df_reconcile.apply(lambda x: 'Match' if x['invoice_no_receipt_no_sqlite'] == x['invoice_no_receipt_no'] and x['invoice_receipt_line_no_sqlite'] == x['invoice_receipt_line_no'] and x['order_item_id_sqlite'] == x['order_item_id'] and x['order_item_sub_id_sqlite'] == x['order_item_sub_id'] else 'Mismatch', axis=1)

# df_reconcile[df_reconcile['status'] == 'Mismatch']

## Describe Statistics

In [32]:
stat_gl_line_sqlite =  pd.DataFrame(select_gl_line_sqlite.describe(include='all'))


# Convert all string values to hashed integers in a copy of the DataFrame
hashed_df = select_gl_line_sqlite.apply(lambda col: col.map(lambda x: hash(x) if isinstance(x, str) else x))

'''
SUM HASHED
'''
# Calculate the sum of each column and convert to a DataFrame row
sum_gl_line_sqlite = hashed_df.sum().to_frame().T
sum_gl_line_sqlite.index = ["sum_hashed"] 

'''
AVG HASHED
'''
avg_gl_line_sqlite = hashed_df.mean().to_frame().T
avg_gl_line_sqlite.index = ["avg_hashed"]

'''
MIN HASHED
'''
min_gl_line_sqlite = hashed_df.min().to_frame().T
min_gl_line_sqlite.index = ["min_hashed"]

'''
MAX HASHED
'''
max_gl_line_sqlite = hashed_df.max().to_frame().T
max_gl_line_sqlite.index = ["max_hashed"]

# Concatenate the statistics DataFrames
stat_select_gl_line_sqlite = pd.concat([stat_gl_line_sqlite, sum_gl_line_sqlite, avg_gl_line_sqlite, min_gl_line_sqlite, max_gl_line_sqlite], axis=0)


In [33]:
# Athena Statistics

stat_gl_line_athena =  pd.DataFrame(select_gl_line_athena.describe(include='all'))


# Convert all string values to hashed integers in a copy of the DataFrame
hashed_df = select_gl_line_athena.apply(lambda col: col.map(lambda x: hash(x) if isinstance(x, str) else x))

'''
SUM HASHED
'''
# Calculate the sum of each column and convert to a DataFrame row
sum_gl_line_athena = hashed_df.sum().to_frame().T
sum_gl_line_athena.index = ["sum_hashed"] 

'''
AVG HASHED
'''
avg_gl_line_athena = hashed_df.mean().to_frame().T
avg_gl_line_athena.index = ["avg_hashed"]

'''
MIN HASHED
'''
min_gl_line_athena = hashed_df.min().to_frame().T
min_gl_line_athena.index = ["min_hashed"]

'''
MAX HASHED
'''
max_gl_line_athena = hashed_df.max().to_frame().T
max_gl_line_athena.index = ["max_hashed"]

# Concatenate the statistics DataFrames
stat_select_gl_line_athena = pd.concat([stat_gl_line_athena, sum_gl_line_athena, avg_gl_line_athena, min_gl_line_athena, max_gl_line_athena], axis=0)



In [34]:
# Compare statistics between SQLite and Athena
stat_reconclie = pd.concat([stat_select_gl_line_sqlite, stat_select_gl_line_athena], axis=1)


#check status if suffix sqlite is match with athena using lambda for each column and row
stat_reconclie['invoice_no_receipt_no_status'] = stat_reconclie.apply(lambda x: 'Match' if x['invoice_no_receipt_no_sqlite'] == x['invoice_no_receipt_no'] else 'Mismatch', axis=1)
stat_reconclie['invoice_receipt_line_no_status'] = stat_reconclie.apply(lambda x: 'Match' if x['invoice_receipt_line_no_sqlite'] == x['invoice_receipt_line_no'] else 'Mismatch', axis=1)
stat_reconclie['order_item_id_status'] = stat_reconclie.apply(lambda x: 'Match' if x['order_item_id_sqlite'] == x['order_item_id'] else 'Mismatch', axis=1)
stat_reconclie['order_item_sub_id_status'] = stat_reconclie.apply(lambda x: 'Match' if x['order_item_sub_id_sqlite'] == x['order_item_sub_id'] else 'Mismatch', axis=1)
stat_reconclie['status'] = stat_reconclie.apply(lambda x: 'Match' if x['invoice_no_receipt_no_sqlite'] == x['invoice_no_receipt_no'] and x['invoice_receipt_line_no_sqlite'] == x['invoice_receipt_line_no'] and x['order_item_id_sqlite'] == x['order_item_id'] and x['order_item_sub_id_sqlite'] == x['order_item_sub_id'] else 'Mismatch', axis=1)


In [38]:
### export to .csv file with sheets for each type of reconcilation
current_date = datetime.now().strftime('%Y%m%d')
# output_path = rf"{pwd}/report/{current_date}_gl_line_reconcile.xlsx"
# with pd.ExcelWriter(output_path) as writer:  
#     df_reconcile.to_excel(writer, sheet_name='Data Reconcile', index=False)
#     stat_reconclie.to_excel(writer, sheet_name='Stat Reconcile', index=True)

output_path_df = rf"{pwd}/report/{current_date}_df_gl_line_reconcile.csv"
output_path_stat = rf"{pwd}/report/{current_date}_stat_gl_line_reconcile.csv"
# Export each dataframe to .csv file
df_reconcile.to_csv(output_path_df,index=False)
stat_reconclie.to_csv(output_path_stat, index=True)
